In [ ]:
%pip install tensorflow cv2 

KeyboardInterrupt: 

In [ ]:
import cv2
import numpy as np
import tensorflow as tf

# Load the TFLite model
interpreter = tf.lite.Interpreter(model_path="best-fp16 (2).tflite")
interpreter.allocate_tensors()

# Get model input details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
input_shape = input_details[0]['shape']

# Load video
cap = cv2.VideoCapture("mixkit-potholes-in-a-rural-road-25208-hd-ready.mp4")  # ⬅️ Replace with your input video path
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Define output video writer
out = cv2.VideoWriter('output_with_detections.avi', cv2.VideoWriter_fourcc(*'XVID'), fps, (width, height))

# Function to preprocess frame for model
def preprocess(frame):
    resized = cv2.resize(frame, (input_shape[2], input_shape[1]))
    input_data = np.expand_dims(resized, axis=0).astype(np.float32)  # Cast to float32 for float model
    input_data /= 255.0  # Normalize to [0, 1] if required by your model
    return input_data

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    input_tensor = preprocess(frame)
    interpreter.set_tensor(input_details[0]['index'], input_tensor)
    interpreter.invoke()

    # NOTE: You must modify the next part depending on your TFLite model's output format
    output_data = interpreter.get_tensor(output_details[0]['index'])[0]

    pothole_count = 0
    for detection in output_data:
        # Adjust this depending on model output. Example assumes YOLO-like [x_center, y_center, width, height, score]
        if detection[4] > 0.5:
            x_center, y_center, w, h = detection[:4]
            x = int((x_center - w / 2) * width)
            y = int((y_center - h / 2) * height)
            w = int(w * width)
            h = int(h * height)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            pothole_count += 1

    # Draw count
    cv2.putText(frame, f'Potholes: {pothole_count}', (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    out.write(frame)

    # Optional: Show live result
    cv2.imshow('Pothole Detection', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Clean up
cap.release()
out.release()
cv2.destroyAllWindows()



In [ ]:
import cv2
import numpy as np
import tensorflow as tf

# Load the TFLite model
interpreter = tf.lite.Interpreter(model_path="best-int8 (1).tflite")
interpreter.allocate_tensors()

# Get model input details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
input_shape = input_details[0]['shape']

# Load video
cap = cv2.VideoCapture("mixkit-potholes-in-a-rural-road-25208-hd-ready.mp4")  # ⬅️ Replace with your input video path
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Define output video writer
out = cv2.VideoWriter('int8.avi', cv2.VideoWriter_fourcc(*'XVID'), fps, (width, height))

# Function to preprocess frame for model
def preprocess(frame):
    resized = cv2.resize(frame, (input_shape[2], input_shape[1]))
    input_data = np.expand_dims(resized, axis=0).astype(np.uint8)  # Cast to uint8 for quantized model
    return input_data

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    input_tensor = preprocess(frame)
    interpreter.set_tensor(input_details[0]['index'], input_tensor) #this is for int8
    interpreter.invoke()

    # NOTE: You must modify the next part depending on your TFLite model's output format
    output_data = interpreter.get_tensor(output_details[0]['index'])[0]

    output_data = interpreter.get_tensor(output_details[0]['index'])[0]

    pothole_count = 0
    for detection in output_data:
        x_center, y_center, w, h, conf, cls = detection
        if conf > 0.5 and int(cls) == 0:  # Assuming pothole class is 0
            x = int((x_center - w / 2) * width)
            y = int((y_center - h / 2) * height)
            w_box = int(w * width)
            h_box = int(h * height)
            cv2.rectangle(frame, (x, y), (x + w_box, y + h_box), (0, 255, 0), 2)
            pothole_count += 1

    # Draw count
    cv2.putText(frame, f'Potholes: {pothole_count}', (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    out.write(frame)

    # Optional: Show live result
    cv2.imshow('Pothole Detection', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Clean up
cap.release()
out.release()
cv2.destroyAllWindows()

c:\Users\supre\anaconda3\Lib\site-packages\tensorflow\lite\python\interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)


In [ ]:
print("Output details:", output_details)
